In [ ]:
import tensorflow as tf
import numpy as np
import os
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical

# Load Dataset
file_path = 'Shakespeare_complete_work.txt'
with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read().lower()

# Preprocessing
text = re.sub(r'[^a-z\s]', '', text)  # Remove punctuation
words = text.split()

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Creating Input Sequences
input_sequences = []
for i in range(1, len(words)):
    n_gram_sequence = words[:i+1]
    encoded_sequence = tokenizer.texts_to_sequences([n_gram_sequence])[0]
    input_sequences.append(encoded_sequence)

# Padding sequences
max_sequence_length = max([len(seq) for seq in input_sequences])
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre')

# Splitting inputs and labels
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = to_categorical(y, num_classes=total_words)

# Model Building
model = Sequential([
    Embedding(total_words, 100, input_length=max_sequence_length-1),
    LSTM(150, return_sequences=True),
    LSTM(100),
    Dense(total_words, activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training
model.fit(X, y, epochs=30, verbose=1)

# Text Generation
def generate_text(seed_text, next_words=50):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Example text generation
seed_text = "to be or not to be"
generated_text = generate_text(seed_text, next_words=50)

# Save Output to File
output_file = 'generated_text_output.txt'
with open(output_file, 'w', encoding='utf-8') as f:
    f.write(generated_text)

print("Generated text saved to", output_file)